# Algoritmo - Localização Bilhetagem (terminais e outros)
### _Autor: Antônio Claudio Dutra Batista_
### _Orientador: Francisco Moraes de Oliveria Neto_
### _Metodologia: Mesclagem por informações coincidentes e critério de tempo_

# 1.0  Leitura das Bases

## 1.1 leitura da base - Bilhetagem (Tratamento Inicial)

In [ ]:
# Arquivos são do mês de novembro de 2018.
import pandas as pd

# Definindo tipos de variáveis com tamanhos menores
tipos_specification = {'id': 'int32', 'linha': 'int32'}

# Lista para armazenar os dataframes de cada grupo de 10 dias
grupos_dias = []

# Loop para ler os arquivos CSV e concatená-los em grupos de 10 dias
for i in range(1, 31, 10):  # Início, fim (não inclusivo), passo
    dias = [pd.read_csv(f'2018-11-{str(j).zfill(2)}.csv', usecols=['id', 'linha', 'nome_linha', 'prefixo_carro', 'nome_cartao', 'sentido_viagem', 'dia', 'momento'], dtype=tipos_specification, sep=',') for j in range(i, i + 10)]
    grupo = pd.concat(dias)
    grupos_dias.append(grupo)

In [ ]:
# Concatenando os grupos de 10 dias
mes_tudo = pd.concat(grupos_dias)

In [ ]:
mes_tudo

In [ ]:
# Caso necessite exportar arquivo da bilhetagem completa
# mes_tudo.to_csv("mes_tudo")

In [1]:
''' FORMATAÇÃO INICIAL '''

' FORMATAÇÃO INICIAL '

In [ ]:
# Momentos em ordem do df mes_c_R!
# mes_tudo.sort_values('momento', inplace = True) # Mostrando DF em ORDEM!

In [ ]:
# Juntando colunas de interesse (dia, momento) em uma só:
mes_tudo["momento"] = mes_tudo['dia'].astype(str) + ' ' + mes_tudo['momento'].astype(str)

In [ ]:
# REMOVENDO COLUNAS DESNECESSÁRIAS!
mes_tudo.drop(['dia'], inplace= True, axis=1)

### 1.1.1 Inspeção inicial

In [ ]:
# Informações da base
mes_tudo.info()

In [ ]:
# Dados nulos 
# mes_tudo.isnull().sum()

In [ ]:
# Verificando linhas utilizadas 
nomes_linhas = mes_tudo['nome_linha']
nomes_linhas.unique()

In [ ]:
# Visualisando dados de identifcação dos veciulos nulos
veiclulos_nao_identificados = mes_tudo[mes_tudo['prefixo_carro'].isnull()]
df_nao_identificados =  veiclulos_nao_identificados['nome_linha'].value_counts()

In [ ]:
df_nao_identificados

In [2]:
''' Ou seja, duas linhas tiveram dados nulos de identificação dos veiculos'''

' Ou seja, duas linhas tiveram dados nulos de identificação dos veiculos'

In [ ]:
# Visualisando dados de identifcação dos nomes das linhas nulos
nome_linha_nao_identificados = mes_tudo[mes_tudo['nome_linha'].isnull()]

In [ ]:
nome_linha_nao_identificados

In [3]:
''' Obs: Nomes da linhas nulos não é um problema, pois se tem o id de identificação delas na coluna linha! '''

' Obs: Nomes da linhas nulos não é um problema, pois se tem o id de identificação delas na coluna linha! '

### 1.1.2 Tratamento da base para localização de validações fora de terminais

In [ ]:
# Memoria liberou 0.2 gb
mes_tudo.info()

In [ ]:
mes_tudo["momento"] = pd.to_datetime(mes_tudo["momento"])

In [ ]:
# Removendo identificação de veiculos com caracteris da coluna de prefixo_carro
mes = mes_tudo[pd.to_numeric(mes_tudo['prefixo_carro'], errors='coerce').notnull()]

In [ ]:
# Trasnformando coluna de prefixo carro em inteiro
mes['prefixo_carro'] = mes['prefixo_carro'].astype('int32')

In [ ]:
mes.info()

In [ ]:
# Última versão do df MES
mes

## 1.2 leitura do base - GPS (Tratamento INICIAL)

In [ ]:
# Definindo o tamanho de cada lote
tamanho_do_lote = 1000000 # 1:22 min tempo

# Definindo os tipos de dados para cada coluna
dtype_specification = {'longitude': 'float32', 'latitude': 'float32', 'vehicle_vehicleid': 'int32'}

# Inicializando uma lista para armazenar os DataFrames de cada lote
lotes = []

# Lendo o arquivo CSV em lotes
for lote in pd.read_csv('Paint112018.csv', usecols=['latitude', 'longitude', 'metrictimestamp', 'vehicle_vehicleid'],
                        dtype=dtype_specification, chunksize=tamanho_do_lote):
    lotes.append(lote)

In [ ]:
# Concatenando todos os lotes em um único DataFrame
GPS_I_H = pd.concat(lotes)

In [ ]:
GPS_I_H

In [ ]:
GPS_I_H.info()

In [ ]:
# FORMATANDO metrictimestamp do (GPS)
GPS_I_H['momento'] = pd.to_datetime(GPS_I_H['metrictimestamp'],format= '%Y%m%d%H%M%S')

In [ ]:
GPS_I_H.drop("metrictimestamp", axis= 1, inplace= True)

In [ ]:
# Renomear coluna de momento de vehicle_vehicleid para id_veiculo:
GPS_I_H.columns=['latitude', 'longitude','id_veiculo', 'momento']

In [ ]:
# Colocando valores em ordem
GPS_I_H.sort_values('momento', inplace= True)

## 1.3 leitura da base dicionário e identificação de validações em terminais

### 1.3.1 leitura e tratamento da base dicionário

In [ ]:
# df', abaixo:
d1 = pd.read_csv('veiculos2018.csv', sep= ';')

In [ ]:
# Mostrando Df
display(d1)

In [4]:
''' Verificamos dados que contem dados com strings '''

' Verificamos dados que contem dados com strings '

In [5]:
''' IREMOS REMOVER ESSES DADOS, POIS POR INSPEÇÃO ELES NÃO INTERFEREM NA LOCALIZAÇÃO DAS VALIDAÇÕES ESPACIALMENTE!'''

' IREMOS REMOVER ESSES DADOS, POIS POR INSPEÇÃO ELES NÃO INTERFEREM NA LOCALIZAÇÃO DAS VALIDAÇÕES ESPACIALMENTE!'

In [ ]:
# Removendo identificação de veiculos com caracteres da coluna de cod_veiculo
d01 = d1[pd.to_numeric(d1['cod_veiculo'], errors='coerce').notnull()]

In [ ]:
d01['cod_veiculo'] = d01['cod_veiculo'].astype('int32')
d01['id_veiculo'] = d01['id_veiculo'].astype('int32')

In [ ]:
# visualizando DUPLICADOS 
d01[d01['cod_veiculo'].duplicated(keep=False)]

In [ ]:
d01.drop_duplicates(subset=['cod_veiculo'], inplace=True)

In [ ]:
# Deixando dados do dicionario que possuem dados associados no GPS!
d01 = d01[d01['id_veiculo'].isin(GPS_I_H['id_veiculo'])]

In [ ]:
d01.reset_index(inplace=True)
d01.drop(columns='index', inplace=True)

In [ ]:
d01

### 1.3.2 Identificação de validações em terminais

In [ ]:
# Verificando dados que estão na bilhetagem (prefixo_carro), mas não estão no dicionário e consequentemente não estão no GPS
dados_apenas_em_mes_ = mes_tudo[~mes_tudo['prefixo_carro'].isin(d01['cod_veiculo'])]

In [ ]:
dados_apenas_em_mes_

In [ ]:
dados_apenas_em_mes_.isnull().sum()

In [6]:
''' OU SEJA, ESSES DADOS NÃO POSSUEM ASSOCIAÇÃO PELO GPS!'''

' OU SEJA, ESSES DADOS NÃO POSSUEM ASSOCIAÇÃO PELO GPS!'

In [ ]:
# Verificando dados que estão na bilhetagem (prefixo_carro), mas não estão no dicionário e consequentemente não estão no GPS
df_terminais_filtro_01 = dados_apenas_em_mes_[dados_apenas_em_mes_['nome_linha'].str.contains( 'TERMINAL')]

#  Por inspeção foi identificado outra nomelcatura para o terminal (ANT.BEZERRA)
df_terminais_filtro_02  = dados_apenas_em_mes_[dados_apenas_em_mes_['nome_linha'].str.contains('TERM.ANT.BEZERRA')]

In [ ]:
# Verificando quantidade de validações em terminais filtro_01
df_terminais_filtro_01['nome_linha'].value_counts()

In [ ]:
# Verificando quantidade de validações em terminais filtro_02
df_terminais_filtro_02['nome_linha'].value_counts()

In [ ]:
# JUNTANDO DADOS DE TERMINAIS EM UMA UNICA BASE 
df_terminais = pd.concat([df_terminais_filtro_01,df_terminais_filtro_02])

In [ ]:
del df_terminais_filtro_01
del df_terminais_filtro_02

In [ ]:
df_terminais.reset_index(inplace=True)
df_terminais.drop(columns='index', inplace=True)

In [ ]:
# Base com TERMINAIS
df_terminais

In [ ]:
df_terminais['nome_linha'].unique()

In [7]:
''' É possivel verificar que o terminal siqueira tinha dados nulos, mas ele é filtrado e fica na varaivel df_terminais, logo a localização dele sera incluso ja nessa varivel! '''

' É possivel verificar que o terminal siqueira tinha dados nulos, mas ele é filtrado e fica na varaivel df_terminais, logo a localização dele sera incluso ja nessa varivel! '

In [8]:
''' Sendo assim é possivel verificar as validações por meio do mesmo id_veiculo e usando o criteiro de tempo, ja os dados dos terminais será usado o metodo de localização desses dados na variavel df_terminais! '''

' Sendo assim é possivel verificar as validações por meio do mesmo id_veiculo e usando o criteiro de tempo, ja os dados dos terminais será usado o metodo de localização desses dados na variavel df_terminais! '

In [9]:
''' PARA PARADAS EM BRT´S NÃO SE TEM O NOME DAS ESTACOES AO LONGO DAS LINHAS!'''

' PARA PARADAS EM BRT´S NÃO SE TEM O NOME DAS ESTACOES AO LONGO DAS LINHAS!'

# 2.0 Junção da bilhetagem com o dicionário 

In [ ]:
# Garantindo uso de validações que possuem relação no GPS
mes_gps = mes_tudo[~mes_tudo['prefixo_carro'].isin(dados_apenas_em_mes_['prefixo_carro'])]

In [ ]:
# Acrescentando dicionário
mes_d1 = mes_gps.merge(d01, left_on= 'prefixo_carro', right_on='cod_veiculo')

In [ ]:
# mes_d1.isnull().sum()

In [ ]:
# Base da Bilhetagem adicionado o dicionario
mes_d1

In [ ]:
mes_d1.info()

In [ ]:
# Criando df somente colunas necessátrias para a integração no tópico 03!
mes_geo = mes_d1.sort_values('momento')

In [ ]:
del mes_d1

In [ ]:
# Reiniciando a contagem dos index
mes_geo.reset_index(inplace=True)

In [ ]:
# REMOVENDO COLUNA DESNECESSÁRIA!
mes_georr = mes_geo.drop(['index', 'cod_veiculo'], axis=1)

In [ ]:
mes_georr['id_veiculo'] = mes_georr['id_veiculo'].astype('int32')

In [ ]:
mes_georr.info()

In [ ]:
del mes_geo

# 3.0 Integração da coordenada aproxima na bilhetagem

## 3.1 Coordenada aproximada do ônibus na bilhetagem (localização de validações fora de terminais e BRT)

In [ ]:
# Coordenada mais proxima do ônibus é extraída da base de GPS e integrada na base da bilhetagem (SOMENTE COLUMAS MOMENTO E ID)
mes_d1_GPS = pd.merge_asof(mes_georr, GPS_I_H, left_on='momento', right_on='momento', by='id_veiculo', direction ='nearest')

In [ ]:
mes_d1_GPS.isnull().sum()

In [ ]:
mes_d1_GPS.info()

In [ ]:
# Base da bilhetagem localizada para dados que possuem associação no GPS!
mes_d1_GPS

In [ ]:
# del GPS_I_H 

## 3.2 Coordenada aproximada dos terminais na bilhetagem

### 3.2.1 Importando dados no formato GTFS

In [ ]:
# Reatribuindo variavel com dados de validações em terminais
df_terminais_validacoes = df_terminais

In [ ]:
del df_terminais

In [ ]:
df_terminais_validacoes.reset_index(inplace=True)

In [ ]:
df_terminais_validacoes.drop(['index'], inplace=True, axis=1)

In [ ]:
# BASE COM VALIDAÇÕES FEITAS NOS TERMINAIS 
df_terminais_validacoes

In [ ]:
df_terminais_validacoes['nome_linha'].unique()

In [ ]:
# Importando base com locais previstos de parada
stops_paradas = pd.read_csv('stops.txt', sep= ',')

In [ ]:
# Removendo colunas desnecessárias do df com paradas 
stops_paradas.drop(columns= ['stop_code', 'stop_desc', 'zone_id', 'stop_url', 'location_type', 'parent_station', 'stop_timezone', 'wheelchair_boarding'], inplace=True)

In [ ]:
stops_paradas

In [10]:
''' POR INSPEÇÃO FOI VERIFICADO QUE A LOCALIZAÇÃO PARA VALIDAÇÃO EM TERMINAIS SÃO NOMEADOS COMO PREÇAS_Nomedoterminal'''

' POR INSPEÇÃO FOI VERIFICADO QUE A LOCALIZAÇÃO PARA VALIDAÇÃO EM TERMINAIS SÃO NOMEADOS COMO PREÇAS_Nomedoterminal'

In [ ]:
# FILTRANDO LOCAIS DAS PRAÇAS NOS TERMINAIS QUE POSSUEM A LOCCALIZAÇÃO
terminais_geometry_ = stops_paradas[stops_paradas['stop_name'].str.contains('TERM')]

In [ ]:
# Reiniciando a contagem do indice 
terminais_geometry_.reset_index(inplace=True)

In [ ]:
terminais_geometry = terminais_geometry_.drop_duplicates(subset=['stop_name'])

In [ ]:
# Dropando coluna desnecessaria
terminais_geometry.drop(columns=['stop_id'], inplace=True)
terminais_geometry.drop(2, inplace= True)

In [ ]:
terminais_geometry.rename(columns={'stop_lat': 'latitude', 'stop_lon': 'longitude'}, inplace=True)

In [ ]:
# Adequando nomes dos terminais para mesclagem
nome_procurado_1 = 'PRAÇA TERMINAL PARANGABA, SN' 
nome_procurado_2 = 'PRAÇA TERMINAL PAPICU, SN' 
nome_procurado_3 = 'PRAÇA TERMINAL MESSEJANA, SN' 
nome_procurado_4 = 'PRAÇA TERMINAL SIQUEIRA, SN' 
nome_procurado_5 = 'PRAÇA TERMINAL CONJUNTO CEARÁ, SN' 
nome_procurado_6 = 'PRAÇA TERMINAL LAGOA, SN' 
nome_procurado_7 = 'PRAÇA TERMINAL ANTONIO BEZERRA, SN'

# Localizando a linha com base no nome e, em seguida, atualizando os valores desejados
terminais_geometry.loc[terminais_geometry['stop_name'] == nome_procurado_1, 'stop_name'] = 'TERMINAL PARANGABA' 
terminais_geometry.loc[terminais_geometry['stop_name'] == nome_procurado_2, 'stop_name'] = 'TERMINAL PAPICU' 
terminais_geometry.loc[terminais_geometry['stop_name'] == nome_procurado_3, 'stop_name'] = 'TERMINAL MESSEJANA' 
terminais_geometry.loc[terminais_geometry['stop_name'] == nome_procurado_4, 'stop_name'] = 'TERMINAL SIQUEIRA'  
terminais_geometry.loc[terminais_geometry['stop_name'] == nome_procurado_5, 'stop_name'] = 'TERMINAL CEARA'  
terminais_geometry.loc[terminais_geometry['stop_name'] == nome_procurado_6, 'stop_name'] = 'TERMINAL LAGOA' 
terminais_geometry.loc[terminais_geometry['stop_name'] == nome_procurado_7, 'stop_name'] = 'TERM.ANT.BEZERRA' 

In [ ]:
terminais_geometry.drop(columns=['index'], inplace=True)

In [ ]:
# BASE COM LOCALIZAÇÃO DOS TERMINAIS 
terminais_geometry

### 3.2.2 Adicioando localização nas validações dos terminais 
''' Com a localização dos terminais ja determindas, é possivel adiconar essa informações nas validações da bilhetagem nesses locais '''

In [ ]:
# MESCLANDO PELO NOME DA LINHA DA BILHETEGEM E O NOME DA PARADA CORRESPONDENTE AS SUAS RESPECTIVAS LINHAS 
mes_terminais_geometry = df_terminais_validacoes.merge(terminais_geometry, left_on= 'nome_linha', right_on='stop_name')

In [ ]:
# Reiniciando a contagem do indice e dropando index
mes_terminais_geometry.reset_index(inplace=True)
mes_terminais_geometry.drop(columns=['index', 'stop_name'], inplace=True)

In [ ]:
# CRIANDO COLUNA ID_LINHA E COD_VEICULO PARA COLOCAR DADOS DE VALIDAÇÕES LOCALIZADOS EM UMA UNICA BASE!
mes_terminais_geometry['id_veiculo'] = 'validacao_terminal'
mes_terminais_geometry['cod_veiculo'] = 'validacao_terminal'

In [ ]:
# Reordenando colunas 
dados_terminais_geo = mes_terminais_geometry[['id', 'linha', 'nome_linha', 'prefixo_carro', 'nome_cartao', 'momento', 'id_veiculo', 'cod_veiculo', 'latitude', 'longitude']]

In [ ]:
del mes_terminais_geometry

In [ ]:
# Base com dados da bilhetagem localizada em terminais
dados_terminais_geo

## 3.3 Coordenada das demais (algumas de BRT´s e outras validações na bilhetagem
''' O pressuposto inicial é que as validações que não possuem um id de identifcação dos veiculos correspondentes no GPS (pelo dicionario) são aquelas que os usuarios validaram no terminais, algumas podem ser de estações de BRT e de casos em que simplismente para os USUARIOS EM QUE A IDENTIFCAÇÃO DO VEICULOS NÃO POSSUI UM CORRESPONDENTE NO GPS, DEVIDO A FALTA DE UMA ASSOCIAÇÃO NO DICIONARIO, como ja foi abordado a localização das validações em terminais, basta realizar uma abordagem com as demais! ''' 

### 3.3.1 Identificando outras validações (possivel estação BRT´s e nos demais casos)
''' Identifcação das validações em possiveis estaçoes de BRT e nos demais casos que não possuiam um veiculo correspondente no GPS'''

In [ ]:
# Reatribuindo variavel com dados que estão na bilhetagem, mas não estão no dicionário e consequentemente não estão no GPS
dados_apenas_em_mes = dados_apenas_em_mes_

In [ ]:
del dados_apenas_em_mes_

In [11]:
''' Foi utilizado a base "mes" pois ela tinha a coluna como numerica para comparação '''

' Foi utilizado a base "mes" pois ela tinha a coluna como numerica para comparação '

In [ ]:
# Base com dados de validaçoes em terminais e em BRT
dados_apenas_em_mes

In [ ]:
# Dropando dados de terminais pois eles ja foram filtrados em uma base propria (OBS: O NOME SERÁ DADOS_BRT, MAS NÃO NECESSPARIAMENTE SÃO TODOS DE BRT´S!), MA
dados_brt_e_outros_ = dados_apenas_em_mes[~dados_apenas_em_mes['nome_linha'].str.contains('TERMINAL', case=False)]
dados_brt_e_outros = dados_brt_e_outros_[~dados_brt_e_outros_['nome_linha'].str.contains('TERM.ANT', case=False)]

In [ ]:
dados_brt_e_outros.reset_index(inplace=True)
dados_brt_e_outros.drop(columns=['index'], inplace=True)

In [ ]:
# BASE COM DADOS DE VALIDAÇÕES SOMENTE PRESENTES NA BILHETAGEM E FORA DOS TERMINAIS (veiculos sem identifcação no dicionario)
dados_brt_e_outros

In [ ]:
# Dados das linhas existentes 
dados_brt_e_outros['linha'].unique()

In [12]:
'''OU SEJA, EXISTEM VALIDAÇÕES DE DIVERSAS LINHAS QUE NÃO TEM UM VEICULO CORRESPONDENTE NO GPS E NÃO SÃO VALIDAÇÕES EM TERMINAIS'''

'OU SEJA, EXISTEM VALIDAÇÕES DE DIVERSAS LINHAS QUE NÃO TEM UM VEICULO CORRESPONDENTE NO GPS E NÃO SÃO VALIDAÇÕES EM TERMINAIS'

### 3.3.2 Associação com o horario programado para localizar as validações em BRT´s e as demais
''' Dados de GTFS, possuem dados de localização das paradas e da viagens programdas, assim é possivem associar para uma mesma linha, e pelo horario mais proximo da validação com o programado de passagem para identificação do possivel local de validação '''

In [ ]:
# Printando novamente dados com locias das paradas
stops_paradas

In [ ]:
# Importando base com horarios previstos de parada  
stop_times = pd.read_csv('stop_times.txt', sep= ',')

# Importando dados com viagens programadas de cada linha 
trips = pd.read_csv('trips.txt', sep=",")

In [ ]:
# Removendo colunas desnecessárias do df com paradas 
stop_times.drop(columns= ['stop_headsign', 'pickup_type', 'drop_off_type', 'shape_dist_traveled'], inplace=True)

In [ ]:
stop_times

In [ ]:
# Acrescetando dados das paradas pela mesmo stop_id no df de tempos previstos de passagem
merge_pardas_time = pd.merge(stop_times, stops_paradas, left_on='stop_id', right_on='stop_id')

In [ ]:
merge_pardas_time

In [ ]:
# Removendo informações desnecessárias do df de dados das viagens
trips_tratado = trips.drop(columns=['service_id', 'trip_headsign', 'trip_short_name', 'direction_id', 'block_id', 'wheelchair_accessible'])

In [ ]:
trips_tratado

In [ ]:
# MESCLANDO BASE COM INFORMAÇÕES DAS PARADAS E TEMPOS DE PASSAGENS COM OS DADSO DAS VIAGENS PROGRAMADAS E AS RESPECTIVAS LINHAS
merge_pardas_time_trips = pd.merge(merge_pardas_time, trips_tratado, left_on='trip_id', right_on='trip_id')

In [ ]:
merge_pardas_time_trips

In [13]:
''' Por inspeção vimos que possuem horarios de chegada superiores a 24 hs, segue na variavel horario_test'''

' Por inspeção vimos que possuem horarios de chegada superiores a 24 hs, segue na variavel horario_test'

In [ ]:
# Verificando dados de tempo maiores que 24 (inspessão)
horario_test = merge_pardas_time_trips[merge_pardas_time_trips.arrival_time>='24:00:00' ]

In [ ]:
horario_test

In [14]:
''' Esses dados são referentes as horarios de 24=00 hs e 25=01hs, foi adotado a substituição de 24 por 00 e 25 por 01 '''

' Esses dados são referentes as horarios de 24=00 hs e 25=01hs, foi adotado a substituição de 24 por 00 e 25 por 01 '

In [ ]:
# transformanmdo dados de horas para string
merge_pardas_time_trips['arrival_time'] = merge_pardas_time_trips['arrival_time'].astype(str)

In [ ]:
# Função para substituição
def substituir_hora_24(hora):
    return hora.replace("24", "00")
def substituir_hora_25(hora):
    return hora.replace("25", "01")

In [ ]:
# Aplicando função para 24
merge_pardas_time_trips['arrival_time'] = merge_pardas_time_trips['arrival_time'].apply(substituir_hora_24)

# Aplicando função para 25
merge_pardas_time_trips['arrival_time'] = merge_pardas_time_trips['arrival_time'].apply(substituir_hora_25)

In [ ]:
# Transformando coluna de chegada programada do veiculo em tipo data
merge_pardas_time_trips['momento_chegada'] = pd.to_datetime(merge_pardas_time_trips['arrival_time'], format='%H:%M:%S')

In [15]:
''' 1900-01-01 é uma data generica! '''

' 1900-01-01 é uma data generica! '

In [ ]:
# Criando coluna de linha (coluna route_id tem essas informações das linhas!)
merge_pardas_time_trips['linha'] = merge_pardas_time_trips['route_id'].astype('int32')

In [ ]:
# Removedno colunas desnecessarias
merge_pardas_time_trips.drop(columns=['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence', 'route_id', 'shape_id'], inplace=True)

In [ ]:
# Base com linhas e os tempos programados de passagem
merge_pardas_time_trips

In [ ]:
merge_pardas_time_trips.isnull().sum()

In [16]:
''' ou seja, existem momentos nulos programados, iremos dropar esses valores!'''

' ou seja, existem momentos nulos programados, iremos dropar esses valores!'

In [ ]:
merge_pardas_time_trips.dropna(subset=['momento_chegada'], inplace=True)

In [ ]:
# colocando df em ordem pelo momento programado de chegada
merge_pardas_time_trips.sort_values('momento_chegada', inplace=True)

In [17]:
''' COM BASE CONTENDO HORARIO PROGRAMADO DE CHEGADA PARA CADA LINHA, É POSSIVEL ASSOCIAR O HORARIO PROGRAMADA MAIS PROXIMO DO HORARIO DE VALIDAÇÃO '''

' COM BASE CONTENDO HORARIO PROGRAMADO DE CHEGADA PARA CADA LINHA, É POSSIVEL ASSOCIAR O HORARIO PROGRAMADA MAIS PROXIMO DO HORARIO DE VALIDAÇÃO '

In [ ]:
# Base dos dados de brt e demais validação sem um veiculo identificado no GPS
dados_brt_e_outros

In [ ]:
# Criando coluna com dados de horario, minuto e segundos de validação 
dados_brt_e_outros['momento_validacao_curto'] = dados_brt_e_outros['momento'].dt.strftime('%H:%M:%S')

In [ ]:
# Transformando coluna de hoario de validação em tipo data
dados_brt_e_outros['momento_validacao_curto'] = pd.to_datetime(dados_brt_e_outros['momento_validacao_curto'], format='%H:%M:%S')

In [ ]:
dados_brt_e_outros # NOVAMENTE 1900-01-01 é uma data generica!

In [ ]:
# colocando df em ordem pelo momento de validação 
dados_brt_e_outros.sort_values('momento_validacao_curto', inplace=True)

In [ ]:
dados_brt_e_outros.isnull().sum()

In [ ]:
# REALIZANDO OPERAÇÃO DE MESCLAGEM PARA LOCALIZAR PELO TEMPO MAIS PROXIMO DE PASSAGEM COM O DE VALIDAÇÃO 
dados_brt_geo_e_outros_ = pd.merge_asof(dados_brt_e_outros, merge_pardas_time_trips, left_on='momento_validacao_curto', right_on='momento_chegada', by='linha', direction ='nearest')

In [ ]:
# Dropando colunas denecessárias
dados_brt_geo_e_outros_.drop(columns=['momento_validacao_curto', 'momento_chegada'], inplace=True)

In [ ]:
# Criando colun id_veiculo e cod_veiculo para mesclagem das bases
dados_brt_geo_e_outros_['id_veiculo'] = 'validacao_brt_outros'
dados_brt_geo_e_outros_['cod_veiculo'] = 'validacao_brt_outros'

In [ ]:
dados_brt_geo_e_outros_.rename(columns={'stop_lat': 'latitude', 'stop_lon': 'longitude'}, inplace=True)

In [ ]:
# Reordenando colunas 
dados_brt_geo_e_outros = dados_brt_geo_e_outros_[['id', 'linha', 'nome_linha', 'prefixo_carro', 'nome_cartao', 'sentido_viagem','momento', 'id_veiculo', 'cod_veiculo', 'latitude', 'longitude']]

In [ ]:
del dados_brt_geo_e_outros_ 

In [ ]:
# BASE LOCALIZADA COM VALIDAÇÕES EM ESTAÇÕES DE BRT E EM CASOS QUE NÃO SE TEM UM VEICULO CORRESPONDENTE NO GPS
dados_brt_geo_e_outros

## 3.4 JUNÇÃO DAS VALIDAÇÕES LOCALIZADAS EM UMA UNICA BASE
''' COM DADOS DE VALIDAÇÕES LOCALIZADOS EM TERMINAIS, EM LINHAS DE BRT´s E EM VALIDAÇÕES EM LINHAS DE ONIBUS CONVENCIONAIS É POSSIVEL JUNTAR ESSAS EM UMA UNICA BASE''' 

In [ ]:
# Concatenando base em um unico df
bilhetagem_localizada = pd.concat([mes_d1_GPS, dados_terminais_geo, dados_brt_geo])

In [ ]:
# Base localizada
bilhetagem_localizada

###### _INTERROPENDO EXECUÇÃO DO SCRIPT!_ 

In [ ]:
import sys
sys.exit()

In [ ]:
# Exportando base localizada para "csv"
# bilhetagem_localizada.to_csv('bilhetagem_localizada_11_2018')

## 3.4.1 Filtrando veiculos em trecho com estações de BRT

### 3.4.1.1 Veiculos com associação do GPS  e que passa na estação

In [ ]:
# Na base dos veiculos em fora de terminais e que possuem um veiculo associado do GPS
validacoes_base_mes_d1_GPS = mes_d1_GPS[(mes_d1_GPS.latitude>=-3.736011)
    & (mes_d1_GPS.longitude>=-38.566622) & (mes_d1_GPS.longitude<=-38.545669) 
                                & (mes_d1_GPS.latitude<=-3.731734)]

In [ ]:
validacoes_base_mes_d1_GPS

In [ ]:
# Linhas localizadas no trecho em questão
validacoes_base_mes_d1_GPS['nome_linha'].unique()

In [ ]:
# Linhas localizadas no trecho em questão
validacoes_base_mes_d1_GPS['linha'].unique()

In [ ]:
# Filtrando linhas especifica
linha_especifica = validacoes_base_mes_d1_GPS[validacoes_base_mes_d1_GPS.linha== 71] # Que passa nas estações

In [ ]:
linha_especifica

In [ ]:
# Veiculos da linha
VEICULOS_GPS = linha_especifica['prefixo_carro'].unique()

In [ ]:
VEICULOS_GPS

### _Pegando veiculo aleatorio_

In [ ]:
aleatorio = linha_especifica['prefixo_carro'].sample(n=1)

In [ ]:
aleatorio

In [ ]:
# Definido validações de veiculo especifico 
val_veiculo = linha_especifica[linha_especifica['prefixo_carro'].isin(aleatorio)]

In [ ]:
val_veiculo

In [ ]:
# Filtrado momento especifico 
val_veiculo_dia = val_veiculo[(val_veiculo.momento>='2018-11-01 07:05:09') & (val_veiculo.momento<='2018-11-02 21:29:06')
                              & (val_veiculo.sentido_viagem<='Volta')]

In [ ]:
val_veiculo_dia

In [ ]:
# !pip install -q folium

In [ ]:
lat = val_veiculo_dia['latitude'].mean()
lon = val_veiculo_dia['longitude'].mean()

In [ ]:
# Criando um mapa centrado em FORTALEZA
import folium
from folium.plugins import MarkerCluster

mapa = folium.Map(location=[lat, lon], zoom_start=5)

# Adicionando marcadores circulares ao mapa
marker_cluster = MarkerCluster().add_to(mapa)

def popup_content(id_veiculo, id_veiculo_clicado):
    cor_popup = 'blue' if id_veiculo != id_veiculo_clicado else 'red'
    return f'<div style="color: {cor_popup};">{id_veiculo}</div>'

for index, row in val_veiculo_dia.iterrows():
    folium.CircleMarker(location=[row['latitude'], row['longitude']],
                        radius=10,
                        color='blue',
                        fill=True,
                        fill_color='green',
                        fill_opacity=0.6,
                        popup=folium.Popup(popup_content(row['id_veiculo'], ''), max_width=3)).add_to(marker_cluster)

In [ ]:
mapa

### 3.4.1.2 Veiculos sem associação do GPS  e que passa na estação
''' Dados de validações fora dos terminais e que não possuem uma assoicação no GPS '''

In [ ]:
# Na base dos veiculos em fora de terminais e que não possuem um veciulo associado no GPS 
validacoes_base_dados_brt_geo = dados_brt_geo[(dados_brt_geo.latitude>=-3.736011)
    & (dados_brt_geo.longitude>=-38.566622) & (dados_brt_geo.longitude<=-38.545669) 
                                & (dados_brt_geo.latitude<=-3.731734)]

In [ ]:
validacoes_base_dados_brt_geo

In [ ]:
validacoes_base_dados_brt_geo['nome_linha'].unique() # Nenhuma linha com nome estação!

In [ ]:
validacoes_base_dados_brt_geo['linha'].unique() 

In [ ]:
# Filtrando linhas especifica
linha_especifica_2 = validacoes_base_dados_brt_geo[validacoes_base_dados_brt_geo.linha== 71]

In [ ]:
linha_especifica_2

In [ ]:
linha_especifica_2['prefixo_carro'] = linha_especifica_2['prefixo_carro'].astype('int64')

In [ ]:
# Veiculos da linha
VEICULOS_GTFS = linha_especifica_2['prefixo_carro'].unique()

In [ ]:
VEICULOS_GTFS

### _Pegando o mesmo veiculo (aletorio do mapa com associação com o gps)_

In [ ]:
aleatorio_2 = linha_especifica_2['prefixo_carro'].sample(n=1)

In [ ]:
aleatorio_2

In [ ]:
# Definido validações de veiculo especifico 
val_veiculo_2 = linha_especifica_2[linha_especifica_2['prefixo_carro'].isin(aleatorio_2)]

In [ ]:
val_veiculo_2

In [18]:
''' OU SEJA, DADA UMA MESMA LINHA, UM VEICULO POSSUE REGISTROS NO GPS E QUE POSSIBILITA A LOCALIZAÇÃO, MAS EXISTE OUTROS VEICULOS SEM ASSOCIAÇÃO QUE SOMENTE É POSSIVEL LOCALIZAR VIA GTFS! '''

' OU SEJA, DADA UMA MESMA LINHA, UM VEICULO POSSUE REGISTROS NO GPS E QUE POSSIBILITA A LOCALIZAÇÃO, MAS EXISTE OUTROS VEICULOS SEM ASSOCIAÇÃO QUE SOMENTE É POSSIVEL LOCALIZAR VIA GTFS! '

In [ ]:
# Filtrado momento especifico 
val_veiculo_dia_2 = val_veiculo_2[(val_veiculo_2.momento>='2018-11-28 05:00:09') & (val_veiculo_2.momento<='2018-11-28 07:16:06')
                                 & (val_veiculo_2.sentido_viagem<='Ida')]

In [ ]:
val_veiculo_dia_2

In [ ]:
lat_2 = val_veiculo_dia_2['latitude'].mean()
lon_2 = val_veiculo_dia_2['longitude'].mean()

In [ ]:
# Criando um mapa centrado em FORTALEZA
import folium
from folium.plugins import MarkerCluster

mapa = folium.Map(location=[lat_2, lon_2], zoom_start=5)

# Adicionando marcadores circulares ao mapa
marker_cluster = MarkerCluster().add_to(mapa)

def popup_content(id_veiculo, id_veiculo_clicado):
    cor_popup = 'blue' if id_veiculo != id_veiculo_clicado else 'red'
    return f'<div style="color: {cor_popup};">{id_veiculo}</div>'

for index, row in val_veiculo_dia_2.iterrows():
    folium.CircleMarker(location=[row['latitude'], row['longitude']],
                        radius=10,
                        color='black',
                        fill=True,
                        fill_color='red',
                        fill_opacity=0.6,
                        popup=folium.Popup(popup_content(row['id_veiculo'], ''), max_width=300)).add_to(marker_cluster)

In [ ]:
mapa

In [19]:
''' OU SEJA, NESSE TRECHO EXISTEM 10 ESTAÇÕES DE BRT E NASA DUAS BASES SOMENTE TEM UMA NA PRIMERIA QUE TEM NOME ESTAÇÃO! E NA BASE QUE NÃO POSSUEM VEICULOS ASSOCIADOS, FORAM DE VALIDAÇÕES EM SOMENTE ALGUAMS LINHAS, E AS DEMAIS VALIDAÇÕES NOS CASOS QUE POSSUEM UM VEICULO CORRESPONDENTE NO GPS, EXISTEM TAMBEM LINHAS QUE PASSAM NESSAS ESTAÇÕES E ELAS POSSUEM NOMES PROPRIOS E NÃO O NOME DA ESTAÇÃO!'''

' OU SEJA, NESSE TRECHO EXISTEM 10 ESTAÇÕES DE BRT E NASA DUAS BASES SOMENTE TEM UMA NA PRIMERIA QUE TEM NOME ESTAÇÃO! E NA BASE QUE NÃO POSSUEM VEICULOS ASSOCIADOS, FORAM DE VALIDAÇÕES EM SOMENTE ALGUAMS LINHAS, E AS DEMAIS VALIDAÇÕES NOS CASOS QUE POSSUEM UM VEICULO CORRESPONDENTE NO GPS, EXISTEM TAMBEM LINHAS QUE PASSAM NESSAS ESTAÇÕES E ELAS POSSUEM NOMES PROPRIOS E NÃO O NOME DA ESTAÇÃO!'

### 3.4.1.3 Verificando lat,long das validações com associação no GPS
''' CASO OS LAT, LONG´S DAS ESTAÇÕES ESTEJAM NO GPS, ENTÃO A IDENTIFICAÇÃO NUMERICA DO VEICULO NA VERDADE É UMA ASSOCIAÇÃO COM A LOCALIZAÇÃO DA ESTAÇÃO '''

In [ ]:
# Id do veiculo selecionado
id_veiculo = val_veiculo['id_veiculo'].unique()

In [ ]:
id_veiculo

In [ ]:
# Filtrando lat,long no GPS desse veiculo da linha especifica
registros_gps = GPS_I_H[GPS_I_H['id_veiculo'].isin(id_veiculo)]

In [ ]:
# Filtrando registros do GPS na região do trecho de estudo
registros_trecho = registros_gps[(registros_gps.latitude>=-3.736011)
    & (registros_gps.longitude>=-38.566622) & (registros_gps.longitude<=-38.545669) 
                                & (registros_gps.latitude<=-3.731734)]

In [ ]:
registros_trecho

In [ ]:
# Base com lat, long das PARADAS (Filtro na região)
stops_trecho = registros_trecho = stops_paradas[(stops_paradas.stop_lat>=-3.736011)
    & (stops_paradas.stop_lon>=-38.566622) & (stops_paradas.stop_lon<=-38.545669) 
                                & (stops_paradas.stop_lat<=-3.731734)]

In [ ]:
stops_trecho

In [ ]:
# Filtrando estações
estacoes = stops_trecho[stops_trecho['stop_name'].str.contains( 'ESTAÇÃO')]

In [ ]:
estacoes

In [20]:
''' PORTANTO, OS REGISTROS NO GPS NESSE TRECHO FUNCIONAM NORMALMENTE E NÃO ASSOCIAM O CODIGO DO VEICULO A LOCAILIZAÇÃO DA ESTAÇÃO! POIS CASO ISSO ACONTECESSE TEERIAM VARIOS REGISTROS REPETIDOS.'''

' PORTANTO, OS REGISTROS NO GPS NESSE TRECHO FUNCIONAM NORMALMENTE E NÃO ASSOCIAM O CODIGO DO VEICULO A LOCAILIZAÇÃO DA ESTAÇÃO! POIS CASO ISSO ACONTECESSE TEERIAM VARIOS REGISTROS REPETIDOS.'

# 4.0 Plotagem da bilhetagem georreferenciada

In [ ]:
# INSTALANDO A BIBLIOTECA GEOPANDAS COM CONDA
# !conda install geopandas -y

# instalando com pip
!pip install-q geopandas

In [ ]:
# mostrar a versão da biblioteca 
# !pip list geopandas

In [ ]:
# instalando a biblioteca matplotlib com conda
# !conda install matplotlib -y

# instalando com pip
!pip install -q matplotlib

In [ ]:
# mostrar a versão da biblioteca 
# !pip list matplotlib

In [ ]:
# importando as bibliotecas
import geopandas as gpd
import matplotlib.pyplot as plt

## 4.2 Leitura de dados espaciais 

In [ ]:
# Ler os dados com geopandas
ceara_muni = gpd.read_file('CE_Municipios_2022.shp')

### 4.2.1 Visualização dos dados 

In [ ]:
# Visualização
# ceara_muni.plot(figsize=(16,14), facecolor='white', edgecolor='black') 
#plt.show()

### 4.2.2 Filtro somente da capital 

In [ ]:
# Salvando somente a capital 'Fortaleza'
fortal = ceara_muni[ceara_muni.NM_MUN=='Fortaleza']
fortal_p = fortal.iloc[0].geometry

In [ ]:
# Mostrar capital 
fortal_p

In [ ]:
# Salvar o geodataframe com formato geojson
filename =  'fortal.json'
fortal.to_file(filename, driver='GeoJSON')

In [ ]:
fortall = gpd.read_file(filename, driver='GeoJSON')

In [ ]:
fortall

### 4.2.3 Modelo de dados espaciais e criação de coluna geometrica 

In [ ]:
# Instalando biblioteca shapely com pip
!pip install -q shapely
# Mostrar versão da biblioteca 
# !pip list shapely

In [ ]:
# importando os objetos geométricos da biblioteca shapely
from shapely.geometry import Point, LineString, Polygon

In [ ]:
# Criando coluna geometerica no df (mes_d1_GPS)
# mes_d1_GPS['geometry'] = None 
# for index, row in mes_d1_GPS.iterrows():
    #mes_d1_GPS.loc[index, 'geometry'] = Point(row.latitude, row.longitude)

In [ ]:
# mes_d1_GPS

In [ ]:
# Criando coluna geometerica no df (mes_d1_GPS)
# mes_d1_GPS['geometry'] = mes_d1_GPS.apply(lambda x: Point((float(x.longitude), float(x.latitude))), axis=1)

In [ ]:
# tipos de dados 
# type(mes_d1_GPS)

In [ ]:
# Criando um GeoDataFrame 
# mes_d1_GPS_geodat_ = gpd.GeoDataFrame(mes_d1_GPS, geometry='geometry')

In [ ]:
# mostar dados
# mes_d1_GPS_geodat_= mes_d1_GPS

In [ ]:
# Salvar o geodataframe com formato geojson
# filename2 =  'mes_d1_GPS_geodat_.json'
# mes_d1_GPS_geodat_.to_file(filename2, driver='GeoJSON')

In [ ]:
# mes_d1_GPS_geodat = gpd.read_file(filename2, driver='GeoJSON')

In [ ]:
# plotando dados (visualização)
# mes_d1_GPS_geodat.plot(figsize=(8,8), facecolor='white', edgecolor='black')

In [ ]:
# type(mes_d1_GPS_geodat)

### 4.2.4 Operação de interseção entre geometrias 

In [ ]:
# Mostrar os dados simultaneamente
# fig, ax = plt.subplots(figsize=(15,15))

# mes_d1_GPS_geodat.plot(ax=ax)
# fortall.plot(ax=ax, facecolor='None', edgecolor='black')

In [ ]:
# VERIFICAMOS QUE EXISTEM DADOS GEORREFERENCIADOS FORA DA MALHA DA CIDADE, PODENDO SER NA SUA MAIORIA REGISTRO DOS VEICULOS
# QUANDO SE DESLOCARAM PARA OUTRAS ZONAS, OU ATÉ MESMO ERROS DO APARELHO DE GPS, COMO MOSTRA NA FIGURA ACIMA UM REGISTRO 
# PRATICAMENTE NO MAR.

In [ ]:
# Realizar operação de interseção entre geometrias
# mes_geo_fortal = mes_d1_GPS_geodat[mes_d1_GPS_geodat.intersects(fortal_p)]

In [ ]:
# mes_geo_fortal.info()

In [ ]:
# NOTAMOS QUE DOS 891038 DADOS DE VALIDAÇÕES APENAS 825386 foram localizadas dentro da malha da cidade, seguimos com estas!

In [ ]:
# Mostrar os dados simultaneamente após interseção
# fig, ax = plt.subplots(figsize=(10,10))

# mes_geo_fortal.plot(ax=ax)
# fortall.plot(ax=ax, facecolor='None', edgecolor='black')

In [ ]:
# Salvando os dados georreferenciados em fortaleza 
# filename3 = 'mes_geo_fortal.json'

# mes_geo_fortal.to_file(filename3, driver='GeoJSON')

### 4.2.5 Mostrando dados no mapa de agrupamento (clusters)

In [ ]:
# Instalando biblioteca folium com pip!
!pip install -q folium

In [ ]:
# importando biblioteca folium e pluging
import folium 
from folium.plugins import FastMarkerCluster

In [ ]:
# Criar mapa de agrupamentos (clusters)
# media_long = mes_geo_fortal['longitude'].mean()
# media_lat = mes_geo_fortal['latitude'].mean()

# mapa = folium.Map(location=[media_lat, media_long])

In [ ]:
# Transformando dados em uma feat geojson
# mc = FastMarkerCluster(mes_geo_fortal[['latitude', 'longitude']])

In [ ]:
# Poligonal da cidade no mapa
limites = folium.features.GeoJson(fortall, style_function=lambda feature:{'color': 'black', 'weight': 2, 'fillOpacity':0.0})

In [ ]:
# Adicionando dados no mapa
# mapa.add_child(mc)
# mapa.add_child(limites)       

In [ ]:
# Salvar o nosso mapa em um arquivo html
# mapa.save('validações_georreferenciadas.html')

### 4.2.6 Mostrando dados no mapa de calor (Heatmap)

In [ ]:
# Importando a plugin HeatMap
from folium.plugins import HeatMap

In [ ]:
# Criando mapa II

# mapa_2 = folium.Map(location=[media_lat, media_long]) 

# Criando heat_map
# heat_map = HeatMap(mes_geo_fortal[['latitude', 'longitude']])     

In [ ]:
# Adicionando no map
# mapa_2.add_child(heat_map)
# mapa_2.add_child(limites)

In [ ]:
# Salvar o nosso mapa em um arquivo html
# mapa_2.save('validações_georreferenciadas_Heatmap_calor.html')

# ------------------------------------------------------------------------